In [1]:
import sys
import numpy as np
sys.path.append('..')

In [2]:
def prune_dataset_by_length(dataset, max_len):
    new_dataset = {}
    for idx in dataset:
        r_mask = dataset[idx]['reactants']['mask']
        r_mask = r_mask[r_mask > 0]
        if len(dataset[idx]['target_main_product']) <= max_len and len(np.unique(r_mask)) == len(r_mask):
            new_dataset[idx] = dataset[idx]
    return new_dataset

In [3]:
import pickle
from utils.draw_utils import draw_gt_reaction
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import SVG, Image
IPythonConsole.molSize = (400,400)

RDKit WARNING: [23:04:01] Enabling RDKit 2019.09.3 jupyter extensions


In [4]:
test_dataset = pickle.load(open('../data/graphs/test.pkl', 'rb'))
train_dataset = pickle.load(open('../data/graphs/train.pkl', 'rb'))
valid_dataset = pickle.load(open('../data/graphs/valid.pkl', 'rb'))

In [5]:
meta = pickle.load(open('../data/graphs/meta.pkl', 'rb'))

In [6]:
meta

{'type': {'AROMATIC', 'DOUBLE', 'SINGLE', 'TRIPLE'},
 'node': {1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  37,
  38,
  39,
  40,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  62,
  63,
  64,
  66,
  68,
  70,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83},
 'features': {'degree': {0, 1, 2, 3, 4, 5, 6, 7},
  'explicit_valence': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 14},
  'hybridization': {0, 1, 2, 3, 4, 5, 6},
  'implicit_valence': {0, 1, 2, 3, 4},
  'is_aromatic': {0, 1},
  'no_implicit': {0, 1},
  'num_explicit_hs': {0, 1, 2, 3, 4, 5, 6},
  'num_implicit_hs': {0, 1, 2, 3, 4},
  'in_ring': {0, 1},
  'num_radical_electrons': {0, 1, 2, 3, 4, 5, 7, 9, 10},
  'formal_charge': {-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6}}}

In [20]:
X, y = [], []
for idx in valid_dataset:
    data = valid_dataset[idx]
    y.append(data['target_main_product'])
    feat = data['reactants']['features']
    feat[:, -1] -= -5
    X.append(feat)
X = np.concatenate(X, axis=-1)
y = np.concatenate(y)

In [21]:
X_test, y_test = [], []
for idx in test_dataset:
    data = test_dataset[idx]
    y_test.append(data['target_main_product'])
    feat = data['reactants']['features']
    feat[:, -1] -= -5
    X_test.append(feat)
X_test = np.concatenate(X_test, axis=-1)
y_test = np.concatenate(y_test)

In [22]:
import catboost

In [23]:
from catboost import CatBoostClassifier

In [25]:
cls = CatBoostClassifier()

In [28]:
cls.fit(X.T, y)

Learning rate set to 0.270532
0:	learn: 0.1794045	total: 441ms	remaining: 7m 20s
1:	learn: 0.0846731	total: 834ms	remaining: 6m 56s
2:	learn: 0.0470799	total: 1.21s	remaining: 6m 43s
3:	learn: 0.0365416	total: 1.58s	remaining: 6m 33s
4:	learn: 0.0313443	total: 1.94s	remaining: 6m 26s
5:	learn: 0.0291943	total: 2.3s	remaining: 6m 20s
6:	learn: 0.0273137	total: 2.67s	remaining: 6m 18s
7:	learn: 0.0245750	total: 3.04s	remaining: 6m 17s
8:	learn: 0.0235093	total: 3.77s	remaining: 6m 54s
9:	learn: 0.0221441	total: 4.22s	remaining: 6m 58s
10:	learn: 0.0208439	total: 4.67s	remaining: 6m 59s
11:	learn: 0.0201780	total: 5.08s	remaining: 6m 58s
12:	learn: 0.0197027	total: 5.46s	remaining: 6m 54s
13:	learn: 0.0193644	total: 5.84s	remaining: 6m 51s
14:	learn: 0.0190442	total: 6.21s	remaining: 6m 48s
15:	learn: 0.0188855	total: 6.58s	remaining: 6m 44s
16:	learn: 0.0187663	total: 6.9s	remaining: 6m 39s
17:	learn: 0.0186419	total: 7.3s	remaining: 6m 38s
18:	learn: 0.0185772	total: 7.69s	remaining: 6m

158:	learn: 0.0169676	total: 57.5s	remaining: 5m 3s
159:	learn: 0.0169676	total: 57.8s	remaining: 5m 3s
160:	learn: 0.0169676	total: 58.1s	remaining: 5m 2s
161:	learn: 0.0169676	total: 58.4s	remaining: 5m 2s
162:	learn: 0.0169676	total: 58.8s	remaining: 5m 2s
163:	learn: 0.0169676	total: 59.2s	remaining: 5m 1s
164:	learn: 0.0169676	total: 59.5s	remaining: 5m 1s
165:	learn: 0.0169676	total: 59.9s	remaining: 5m
166:	learn: 0.0169676	total: 1m	remaining: 5m
167:	learn: 0.0169676	total: 1m	remaining: 4m 59s
168:	learn: 0.0169676	total: 1m	remaining: 4m 59s
169:	learn: 0.0169676	total: 1m 1s	remaining: 4m 58s
170:	learn: 0.0169676	total: 1m 1s	remaining: 4m 58s
171:	learn: 0.0169676	total: 1m 1s	remaining: 4m 57s
172:	learn: 0.0169676	total: 1m 2s	remaining: 4m 57s
173:	learn: 0.0169676	total: 1m 2s	remaining: 4m 56s
174:	learn: 0.0169676	total: 1m 2s	remaining: 4m 56s
175:	learn: 0.0169676	total: 1m 3s	remaining: 4m 55s
176:	learn: 0.0169676	total: 1m 3s	remaining: 4m 55s
177:	learn: 0.016

312:	learn: 0.0169539	total: 1m 49s	remaining: 3m 59s
313:	learn: 0.0169539	total: 1m 49s	remaining: 3m 59s
314:	learn: 0.0169539	total: 1m 49s	remaining: 3m 58s
315:	learn: 0.0169539	total: 1m 50s	remaining: 3m 58s
316:	learn: 0.0169539	total: 1m 50s	remaining: 3m 58s
317:	learn: 0.0169539	total: 1m 50s	remaining: 3m 57s
318:	learn: 0.0169539	total: 1m 51s	remaining: 3m 57s
319:	learn: 0.0169539	total: 1m 51s	remaining: 3m 57s
320:	learn: 0.0169539	total: 1m 51s	remaining: 3m 56s
321:	learn: 0.0169539	total: 1m 52s	remaining: 3m 56s
322:	learn: 0.0169539	total: 1m 52s	remaining: 3m 56s
323:	learn: 0.0169539	total: 1m 52s	remaining: 3m 55s
324:	learn: 0.0169539	total: 1m 53s	remaining: 3m 55s
325:	learn: 0.0169539	total: 1m 53s	remaining: 3m 54s
326:	learn: 0.0169539	total: 1m 53s	remaining: 3m 54s
327:	learn: 0.0169539	total: 1m 54s	remaining: 3m 54s
328:	learn: 0.0169539	total: 1m 54s	remaining: 3m 53s
329:	learn: 0.0169539	total: 1m 54s	remaining: 3m 53s
330:	learn: 0.0169539	total:

465:	learn: 0.0169539	total: 2m 39s	remaining: 3m 3s
466:	learn: 0.0169539	total: 2m 40s	remaining: 3m 2s
467:	learn: 0.0169539	total: 2m 40s	remaining: 3m 2s
468:	learn: 0.0169539	total: 2m 40s	remaining: 3m 2s
469:	learn: 0.0169539	total: 2m 41s	remaining: 3m 1s
470:	learn: 0.0169539	total: 2m 41s	remaining: 3m 1s
471:	learn: 0.0169539	total: 2m 41s	remaining: 3m 1s
472:	learn: 0.0169539	total: 2m 42s	remaining: 3m
473:	learn: 0.0169539	total: 2m 42s	remaining: 3m
474:	learn: 0.0169539	total: 2m 42s	remaining: 2m 59s
475:	learn: 0.0169539	total: 2m 43s	remaining: 2m 59s
476:	learn: 0.0169539	total: 2m 43s	remaining: 2m 59s
477:	learn: 0.0169539	total: 2m 44s	remaining: 2m 59s
478:	learn: 0.0169539	total: 2m 44s	remaining: 2m 58s
479:	learn: 0.0169539	total: 2m 44s	remaining: 2m 58s
480:	learn: 0.0169539	total: 2m 45s	remaining: 2m 58s
481:	learn: 0.0169539	total: 2m 45s	remaining: 2m 57s
482:	learn: 0.0169539	total: 2m 45s	remaining: 2m 57s
483:	learn: 0.0169539	total: 2m 46s	remaini

618:	learn: 0.0169539	total: 3m 31s	remaining: 2m 9s
619:	learn: 0.0169539	total: 3m 31s	remaining: 2m 9s
620:	learn: 0.0169539	total: 3m 31s	remaining: 2m 9s
621:	learn: 0.0169539	total: 3m 32s	remaining: 2m 8s
622:	learn: 0.0169539	total: 3m 32s	remaining: 2m 8s
623:	learn: 0.0169539	total: 3m 32s	remaining: 2m 8s
624:	learn: 0.0169539	total: 3m 33s	remaining: 2m 7s
625:	learn: 0.0169539	total: 3m 33s	remaining: 2m 7s
626:	learn: 0.0169539	total: 3m 33s	remaining: 2m 7s
627:	learn: 0.0169539	total: 3m 33s	remaining: 2m 6s
628:	learn: 0.0169539	total: 3m 34s	remaining: 2m 6s
629:	learn: 0.0169539	total: 3m 34s	remaining: 2m 6s
630:	learn: 0.0169539	total: 3m 34s	remaining: 2m 5s
631:	learn: 0.0169539	total: 3m 35s	remaining: 2m 5s
632:	learn: 0.0169539	total: 3m 36s	remaining: 2m 5s
633:	learn: 0.0169539	total: 3m 36s	remaining: 2m 4s
634:	learn: 0.0169539	total: 3m 36s	remaining: 2m 4s
635:	learn: 0.0169539	total: 3m 36s	remaining: 2m 4s
636:	learn: 0.0169539	total: 3m 37s	remaining:

772:	learn: 0.0169539	total: 4m 22s	remaining: 1m 17s
773:	learn: 0.0169539	total: 4m 22s	remaining: 1m 16s
774:	learn: 0.0169539	total: 4m 23s	remaining: 1m 16s
775:	learn: 0.0169539	total: 4m 23s	remaining: 1m 16s
776:	learn: 0.0169539	total: 4m 23s	remaining: 1m 15s
777:	learn: 0.0169539	total: 4m 24s	remaining: 1m 15s
778:	learn: 0.0169539	total: 4m 24s	remaining: 1m 15s
779:	learn: 0.0169539	total: 4m 24s	remaining: 1m 14s
780:	learn: 0.0169539	total: 4m 25s	remaining: 1m 14s
781:	learn: 0.0169539	total: 4m 25s	remaining: 1m 14s
782:	learn: 0.0169539	total: 4m 25s	remaining: 1m 13s
783:	learn: 0.0169539	total: 4m 26s	remaining: 1m 13s
784:	learn: 0.0169539	total: 4m 26s	remaining: 1m 12s
785:	learn: 0.0169539	total: 4m 26s	remaining: 1m 12s
786:	learn: 0.0169539	total: 4m 27s	remaining: 1m 12s
787:	learn: 0.0169539	total: 4m 27s	remaining: 1m 11s
788:	learn: 0.0169539	total: 4m 27s	remaining: 1m 11s
789:	learn: 0.0169539	total: 4m 28s	remaining: 1m 11s
790:	learn: 0.0169539	total:

928:	learn: 0.0169489	total: 5m 10s	remaining: 23.8s
929:	learn: 0.0169489	total: 5m 11s	remaining: 23.4s
930:	learn: 0.0169489	total: 5m 11s	remaining: 23.1s
931:	learn: 0.0169489	total: 5m 11s	remaining: 22.7s
932:	learn: 0.0169489	total: 5m 11s	remaining: 22.4s
933:	learn: 0.0169489	total: 5m 12s	remaining: 22.1s
934:	learn: 0.0169489	total: 5m 12s	remaining: 21.7s
935:	learn: 0.0169489	total: 5m 12s	remaining: 21.4s
936:	learn: 0.0169489	total: 5m 12s	remaining: 21s
937:	learn: 0.0169489	total: 5m 12s	remaining: 20.7s
938:	learn: 0.0169489	total: 5m 13s	remaining: 20.3s
939:	learn: 0.0169489	total: 5m 13s	remaining: 20s
940:	learn: 0.0169489	total: 5m 13s	remaining: 19.7s
941:	learn: 0.0169489	total: 5m 13s	remaining: 19.3s
942:	learn: 0.0169489	total: 5m 13s	remaining: 19s
943:	learn: 0.0169489	total: 5m 14s	remaining: 18.6s
944:	learn: 0.0169489	total: 5m 14s	remaining: 18.3s
945:	learn: 0.0169489	total: 5m 14s	remaining: 18s
946:	learn: 0.0169489	total: 5m 14s	remaining: 17.6s
9

In [31]:
cls.feature_importances_

array([ 1.71374707,  1.49231649,  2.24398115,  0.77387636,  1.57927848,
       86.9274805 ,  2.07996119,  0.41657858,  0.73428036,  0.37122096,
        1.66727886])

In [42]:
idx = 1

In [43]:
data = test_dataset[idx]

In [44]:
for i, j in zip(data['reactants']['features'][5], data['target_main_product']):
    print(i, j, i==j)

1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
0 0 True
0 0 True
0 0 True
0 0 True
0 0 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
1 1 True
0 0 True
0 0 True
0 0 True
0 0 True
0 0 True
0 0 True
0 0 True
0 0 True
0 0 True
0 0 True
0 0 True
0 0 True
0 0 True
0 0 True
1 1 True
0 0 True
0 0 True
0 0 True
1 1 True
11 1 False
